In [ ]:
## This is for plotting reservoir storage
%matplotlib widget
import csv
import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.widgets import Slider, Button, RadioButtons
import numpy as np

reservoirs = [
    'castaic',
    'del_valle',
    'feather',
    'pyramid',
    'silverwood',
    'thermalito',
]

cdec_stations = dict(
    castaic=['CAS'],
    del_valle=['DLV'],
    feather=['ANT', 'DAV', 'FRD', 'ORO'],
    pyramid=['PYM'],
    silverwood=['SIV'],
    thermalito=['TAB', 'TFR'],
)

def get_month_code(month_str):
    """Returns the month integer based off the provided
    three-letter code, i.e. get_month_code('Mar') = 3
    """
    
    if not len(month_str) == 3:
        print('Input must be three-letter code.')
        return
    
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    return months.index(month_str) + 1
        

def load_monthly_reservoir_data(reservoir, station):
    """Loads the monthly reservoir storage data available
    for the provided reservoir and station.
    """
    if not reservoir in reservoirs:
        print('Reservoir not found')
        return
    
    if not station in cdec_stations[reservoir]:
        print('Station ID not found')
        return
    
    dates=[]
    storage=[]
    
    filename = '../data/'+reservoir+'/cdec_'+station+'_monthly.csv'
    
    with open(filename) as data:
        reader = csv.reader(data)
        num_empty = 0
        for row in reader:
            # Date is in format Month-YY
            date_parts = row[0].split('-')
            
            #
            if not len(date_parts[0]) == 3:
                continue
            month = get_month_code(date_parts[0])
            year = int(date_parts[1])
            
            # Year is two-digit so make it four
            if year >=0 and year <= 19:
                year = 2000 + year
            else:
                year = 1900 + year

            # Create a date object from parts
            date = datetime.date(year, month, 1)

            # Add the date and storage to arrays
            # These are all the different ways CDEC stores an empty cell
            if row[1] == '' or row[1] == 'm' or row[1] == '--':
                continue
            else:
                dates.append(date)
                storage.append(float(row[1]))
    
    return np.array(dates), np.array(storage)

default_reservoir='feather'
default_station = 'ORO'

dates, storage = load_monthly_reservoir_data(default_reservoir, default_station)
min_date = dates[0]
max_date = dates[-1]

fig = plt.figure(figsize=(10, 6), dpi=80)
ax = fig.gca()

# Shrink the plot by 25% towards upper-right corner
plt.subplots_adjust(left=0.25, bottom=0.25)
l, = plt.plot(dates, storage, linewidth=2, color='red')

ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
ax.set_title(default_reservoir + ' (' + default_station + ')')

axcolor = 'lightgoldenrodyellow'
axyear_start = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)
axyear_end = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)

syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')

def find_index(dates, year):
    index = 0
    for date in dates:
        if date.year == int(year):
            return index
        index += 1
    return -1

def update(val):
    year_start = syear_start.val
    year_end = syear_end.val
    if not year_start < year_end:
        return
    s_index = find_index(dates, year_start)
    e_index = find_index(dates, year_end + 1)
    l.set_xdata(dates[s_index:e_index])
    l.set_ydata(storage[s_index:e_index])
    ax.relim()
    ax.autoscale_view()
    plt.draw()
syear_start.on_changed(update)
syear_end.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

def reset(event):
    global syear_start, syear_end, station_radio, reservoir_radio, dates, storage
    reservoir = default_reservoir
    station = default_station
    # Load new data
    dates, storage = load_monthly_reservoir_data(reservoir, station)
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(storage)
    ax.set_title(reservoir + ' (' + station + ')')
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    # Reset radio buttons
    rax.clear()
    reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))
    reservoir_radio.on_clicked(set_reservoir)
    stationax.clear()
    station_radio = RadioButtons(stationax, cdec_stations[default_reservoir], active=cdec_stations[default_reservoir].index(default_station))
    station_radio.on_clicked(set_station)
    fig.canvas.flush_events()
button.on_clicked(reset)

rax = plt.axes([0.025, 0.6, 0.15, 0.3], facecolor=axcolor)
reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))

stationax = plt.axes([0.025, 0.25, 0.15, 0.15], facecolor=axcolor)
station_radio = RadioButtons(stationax, cdec_stations[default_reservoir], active=cdec_stations[default_reservoir].index(default_station))

def set_station(label):
    global syear_start, syear_end, dates, storage
    reservoir = reservoir_radio.value_selected
    station = label
    # Load new data
    dates, storage = load_monthly_reservoir_data(reservoir, station)
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(storage)
    ax.set_title(reservoir + ' (' + station + ')')
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    fig.canvas.flush_events()
station_radio.on_clicked(set_station)

def set_reservoir(label):
    global syear_start, syear_end, station_radio, dates, storage
    reservoir = label
    station = cdec_stations[reservoir][0]
    # Load new data
    dates, storage = load_monthly_reservoir_data(reservoir, station)
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(storage)
    ax.set_title(reservoir + ' (' + station + ')')
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    # Reset site no selector
    stationax.clear()
    station_radio = RadioButtons(stationax, cdec_stations[reservoir], active=0)
    station_radio.on_clicked(set_station)
    fig.canvas.flush_events()
reservoir_radio.on_clicked(set_reservoir)

plt.show()